# Powertrain for UAV

In [343]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

### The battery
- Something is off with the battery, need to double check the series / parallel configuration

In [5283]:
class DiscreteBattery:
    def __init__(self, cells: DiscreteBatteryCell, wiring):
        self.cells = cells
        self.wiring = wiring
        self.v0 = self.cells[0].v0 if wiring == "parallel" else sum(cell.v0 for cell in self.cells)
        self.v0 = round(self.v0, 3)
        self.soc = 1.0
        self.ocv = self.v0
        self.eod = self.cells[0].eod if wiring == "parallel" else sum(cell.eod for cell in celf.cells)
        
    def step(self, dt, current):
        if(self.wiring == "parallel"):
            for cell in self.cells:
                cell.step(dt, current)
            self.soc = sum(cell.soc for cell in self.cells) / 3
            self.ocv = sum(cell.ocv for cell in self.cells) / 3

class DiscreteBatteryCell:
    def __init__(self, **kwargs):
        if(len(kwargs) == 0):
            self.soc_ocv = np.zeros((1,1))
            self.R0_degradation = np.zeros((1,1))
            self.Q_degradation = np.zeros((1,1))
        else:
            self.soc_ocv = kwargs["soc_ocv"]
            self.R0_degradation = kwargs["R0_degradation"]
            self.Q_degradation = kwargs["Q_degradation"]
        self.soc = 1.0
        self.Ir = 0
        self.h = 0
        self.M0 = .0019
        self.M = .0092
        self.R0 = .0112
        self.R = 2.83e-4
        self.Q = 3.8695
        self.n = .9987
        self.G = 163.4413
        self.v0 = 4.2
        self.eod = 3.04
        self.RC = 3.6572
        self.ocv = self.v0
        
    def get_ocv(self):
        if(self.soc < 0.0):
            self.soc = 0
        elif(self.soc > 1.0):
            self.soc = 1.0
        idx = int(np.ceil(self.soc*100))
        if(idx > 101):
            idx = 101
        elif(idx < 1):
            idx = 1
        return self.soc_ocv[idx]
        
    def step(self, dt, current):
        RC = np.exp(-dt/abs(self.RC))
        H = np.exp(-abs(self.n*current*self.G*dt/(3600*self.Q)))
        self.Ir = RC*self.Ir + (1-RC)*current
        self.h = H*self.h + (H-1)*np.sign(current)
        self.soc = self.soc - self.n*current/3600/self.Q
        self.ocv = self.get_ocv() + self.M*self.h + self.M0*np.sign(current) - self.R*self.Ir - self.R0*current
        
    def add_profile(self,**profile):
        self.soc_ocv = profile["soc_ocv"]
        self.R0_degradation = profile["R0_degradation"]
        self.Q_degradation = profile["Q_degradation"]
        
        
def get_battery_curves(soc_ocv_file, R0_degradation_file, Q_degradation_file):
    """
        @brief: gets the degradation profile (predefined curves) for the battery
        
        @input:
            soc_ocv_file: a csv file containing the soc_ocv relationship as a column vector
            R0_degradation_file: a csv file containing the R0 degradation curve as a column vector
            Q_degradation_file: a csv file containing the Q degradation curve as a column vector
        
        @output:
            a dictionary mapping of degradation curves with ["soc_ocv", "R0_degradation", "Q_degradation"] keys
    """    
    soc_ocv = []
    R0_degradation = []
    Q_degradation = []
    
    with open(soc_ocv_file, newline='') as f:
        soc_ocv = list(csv.reader(f))
    soc_ocv = np.asarray(soc_ocv).astype(np.float)
    
    with open(R0_degradation_file, newline='') as f:
        R0_degradation = list(csv.reader(f))
    R0_degradation = np.asarray(R0_degradation).astype(np.float)
    
    with open(Q_degradation_file, newline='') as f:
        Q_degradation = list(csv.reader(f))
    Q_degradation = np.asarray(Q_degradation).astype(np.float)
    
    return {"soc_ocv": soc_ocv, "R0_degradation": R0_degradation, "Q_degradation": Q_degradation}

battery_curves = get_battery_curves('soc_ocv.csv', 'R0_degradation.csv', 'Q_degradation.csv')

### test the cells and battery

In [5306]:
cells = []
for i in range(0,3):
    cells.append(DiscreteBatteryCell(**battery_curves))
battery = DiscreteBattery(cells, "parallel")
cell = DiscreteBatteryCell(**battery_curves)

#### simulate the cell
- dt = 1 second, current = 3.8695
- the cell capacity is 3.8695 mAh, so, i should be ~3600 
- the cell asymptotically approaches the eod, so have to stop slightly above

In [5307]:
i = 0
while(cell.ocv > (cell.eod * 1.005)):
    cell.step(1, 3.8695)
    i += 1
print(cell.ocv)
print(i)

[3.04926653]
3569


#### simulate the battery
- same as above, since the battery is configured as parallel cells

In [5308]:
i = 0
while(battery.ocv > (battery.eod * 1.005)):
    battery.step(1, 3.8695)
    i += 1
print(battery.ocv)
print(i)

[3.04926653]
3569
